# Covariant encoding via RF states, based on 5-qubit code

# RF state changed to be 2N qubit sine state
- Only need to change the 3rd part in functions: p(h|g)
- Calculate the entanglement error and the worst-case error

# Note: In this program, we calculate entanglement fidelity
- For qubit covariant channel:
    1. entanglement error = 1-Fent'
    2. worst case error <= 2 * entanglement error

# 1. Used modules and some matrices

In [4]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt  # For plot
from scipy.optimize import curve_fit  # For fitting
from scipy.stats import unitary_group  # Generate random unitary matrix


# get measure outcome via optimization, with the help of "paddle quantum"
import paddle
from paddle_quantum.circuit import UAnsatz  # Create Quantum Circuit
from paddle_quantum.state import vec, vec_random  # Generate Quantum state, (vector)
from paddle_quantum.state import density_op, density_op_random, completely_mixed_computational  # Generate Quantum state, (matrix)
from paddle_quantum.utils import dagger  # Take dagger for paddle.tensor
from paddle import matmul, trace  # Calculate inner product and trace for paddle.tensor


In [5]:
# Some matrices

# Bell state -- np ket
bell_state = np.array([[1,0,0,1]]) / (2**0.5)
bell_state = bell_state.conj().T

# Pauli matrices
pauli_x = np.array([[0.0, 1.0], [1.0, 0.0]])
pauli_y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
pauli_z = np.array([[1.0, 0.0], [0.0, -1.0]])

# 2. Generate random SU(2) matrix

In [6]:
# 2. Generate random SU(2) martrix: -- U2_random_Euler_mod()

def Rz(alpha):
    # output U：np.array -- rotation around Z axis with angle alpha
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.e ** (-1j * alpha/2)
    U[1][1] = np.e ** (1j * alpha/2)
    
    return U

def Ry(alpha):
    # output U：np.array -- rotation around Y axis with angle alpha
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.cos(alpha/2)
    U[0][1] = -np.sin(alpha/2)
    U[1][0] = np.sin(alpha/2)
    U[1][1] = np.cos(alpha/2)
    
    return U

def U2_Euler(alpha, beta, gamma):
    # output U：np.array -- rotation with Euler angle representation
    # U(alpha, beta, gamma) = Z(gamma) @ Y(beta) @ Z(alpha)
    # Note: alpha, gamma \in [0,2pi], beta \in [0,pi]
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    return U



def random_sinx(a, b):
    # Gerenrate distribution sin(x)dx, x \in [a,b]
        # Note：sin(x) should be > 0 for x \in [a,b]
    # Generate random (x,y), x \in [a,b], y \in [0, maxy]
        # for (xi, yi), if 0<yi<f(xi), output xi
    ymax = 1  # sin(x) always smaller than 1
    while 1:
        x = random.uniform(a, b)
        y = random.uniform(0, 1)
        if y < np.sin(x):
            break
    return x,y

def U2_random_Euler_mod():
    # output U：np.array -- rotation with Euler angle representation
    # beta in distribution sin(beta)d(beta), alpha, gamma are uniform
    alpha = random.uniform(0, 2*np.pi)
    beta, anc = random_sinx(0, np.pi)
    gamma = random.uniform(0, 2*np.pi)
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    
    return U

# 3.From 2N qubit sine state (after Ug), measure and get final Uh (this part needs to be modified)
- random_fromg_geth(Ug) -- random_fromg_geth_largeRF(Ug)
    1. Given N,d, get the lambda_via and q_lambda -- Here we use sine state
    2. Calculate p(h|g) with this RF state
    3. Get a list of Uhi with p(h|g)
    4. Uh is derived without optimization, but from p(h|g) directly (as for this RF state, we only get one measure outcome)

In [7]:
# Given N,d, get the lambda_via and q_lambda -- Here we use sine state
# The coefficients for the RF state -- sine state
def q_lambda(N, d):
    # Input: d:SU(d) -- now we only deal with d=2
            # N: N RF qubits
    # Output：2 list
        # lambda_via：the viable lambda: for SU(2): lambda2 \in (1, N // 2 + 1), lambda_via = lambda1 = N - lambda2
        # q_via：The coefficient for lambda_1
    lambda_via = []
    q_via = []
    J = N / 2
    if d == 2:
        for lambda_2 in range(1, N // 2 + 1):
            lambda_1 = N - lambda_2
            j = (lambda_1 - lambda_2) / 2
            q = np.sin( np.pi * j  / J ) ** 2 / (2 * j + 1)
            if j != 0:
                lambda_via.append(lambda_1)  # keep this lambda_1 as lambda_via
                q_via.append(q)

    return lambda_via, q_via

# Normalization
def q_lambda_nor(lambda_via, q_via):
    total_q = 0
    for q in q_via:
        total_q += q
    for i in range(len(q_via)):
        q_via[i] = q_via[i] / total_q
    return lambda_via, q_via

# p(h|g) with sine state
def pgh_largeRF(N, d, Ug, Uh):
    # Input: d:SU(d) -- now we only deal with d=2
            # N: N RF qubits
    # p(g|h) = | \Sum_{lambda_i \in lambda_via} sqrt{q_lambda_i} * Tr[(Ug * Uh^{dag, j})]  |^2
        # j = (lambda_i1 - lambda_i2) / 2
        # Tr[U^{1/2}] = 2 cos(\phi/2)
        # Tr[U^{j}] = sin(\phi * (2j + 1)/2) / sin(\phi/2)
    
    # 1. The normalized lambda_via, q_via
    lambda_via, q_via = q_lambda(N, d)
    lambda_via, q_via = q_lambda_nor(lambda_via, q_via)
    
    # 2. phi
    phi = np.arccos(np.real(np.trace(Ug @ Uh.conj().T)) / 2) * 2  # In [0, 2*Pi]
    if phi == 0:
        phi = phi + 0.000001
    pghi = 0
    for i in range(len(lambda_via)):
        lambda_i1 = lambda_via[i]
        lambda_i2 = N - lambda_i1
        j = (lambda_i1 - lambda_i2) / 2
        pghi += np.sqrt(q_via[i]) * np.sin(phi * (2*j + 1)/2) / np.sin(phi/2)
    pgh = pghi ** 2
    
    return pgh


- Uh is derived without optimization, but from p(h|g) directly

In [8]:
# From Ug get Uh with distribution p(h|g)
def random_fromg_geth_largeRF(N, d, Ug):
    # Input: Ug: np.matrix, distribution, N: RF qubits number, d = 2
    # Output: Uh:np.matrix: distribution -- p(h|g)
    pgh_max = pgh_largeRF(N, d, -np.eye(2), -np.eye(2))
    while 1:
        Uh = U2_random_Euler_mod()
        # change to SU(2)
        coeUh = np.sqrt(np.linalg.det(Uh))
        Uh = coeUh.conj() * Uh
        pgh = random.uniform(0, pgh_max)
        fgh = pgh_largeRF(N, d, Ug, Uh)
        if pgh < fgh:
            break
    return Uh

# Main function:
def From_g_get_hath_largeRF(N, d, Ug):
    Uh = random_fromg_geth_largeRF(N, d, Ug)
    return Uh
    

# The modified part end, use From_g_get_hath_largeRF(N, d, Ug) for the following

# 4. About coding -- functions related to 5-qubit code

In [9]:
# Some related functions for encoding, decoding

# Binary to Decimal ---- for encoding
def TwoToTen(a_two):
    # Input: Binary string:  a_two
    # Output: Decimal number: a_ten
    a_ten = 0
    for i in range(len(a_two)):
        new = int(a_two[len(a_two)-i-1])  # Get the last "i+1"-th number
        a_ten = a_ten + new * (2**i)
    
    return a_ten

# Decimal to Binary ---- for decoding
def TenToTwo(a_ten, n):
    # Input: Decimal number: a_ten, n: the length of the output string
    # Output: Binary string:  a_two
    a_two = ''
    while a_ten:
        new = a_ten % 2
        new = str(new)
        a_two += new
        a_ten = a_ten // 2
    if len(a_two) < n:
        for i in range(n - len(a_two)):
            a_two += '0'
    a_two = a_two[::-1]
    
    return a_two

# Get the measure probability and output state for the 5-qubit code
# state is in order: 4 measure qubits \otimes 5 code qubits \otimes 1 ancilla qubit
def MeasFiveQ(final_state, basis):
    # Input: final_state: 10-qubit code state after noise (numpy.matrix), basis: wanted measure outcome 
        # 0 for 0000, 1 for 0001,... 15 for 1111
    # Output: The measure probability and the remaining 6-qubit state: 5 code qubits \otimes 1 ancilla qubit (numpy.matrix)
        # pr, meas_prob_basis
    meas_prob_basis = np.kron(vec(basis, 4), np.eye(2**6)) @ final_state @ np.kron(vec(basis, 4).conj().T, np.eye(2**6))
    pr = np.real(meas_prob_basis.trace())
    if pr > 0:  # pr should not be 0
        meas_prob_basis = meas_prob_basis / pr  # 归一化
    
    return pr, meas_prob_basis

# Circuit for Stabilizer measure for 5-qubit code (in paddle quantum form)
def stabilizer_meas(cir):
    # Input: 10-qubit initial circuit
    # Four initial H gates on measure qubits
    for i in range(4):
        cir.h(i)
    # 1：XZZXI
    cir.cnot([0,4])
    cir.cz([0,5])
    cir.cz([0,6])
    cir.cnot([0,7])
    # 2：IXZZX
    cir.cnot([1,5])
    cir.cz([1,6])
    cir.cz([1,7])
    cir.cnot([1,8])
    # 3：XIXZZ
    cir.cnot([2,6])
    cir.cz([2,7])
    cir.cz([2,8])
    cir.cnot([2,4])
    # 4：ZXIXZ
    cir.cnot([3,7])
    cir.cz([3,8])
    cir.cz([3,4])
    cir.cnot([3,5])
    # Four final H gates on measure qubits
    for i in range(4):
        cir.h(i)
        
    return None

# Do correction in decoding according to measure outcomes
def correct_accoring_tomeas(meas_out, outstate):
    # Input: meas_out: measure outcome (string)
            # outstate (np.matrix), 5 code \otimes 1 anc
    # Output: final_state: state after correction (np.matrix)

    if meas_out == '0000':
        U_cor = np.eye(2**6)
    elif meas_out == '0001':
        U_cor = np.kron(pauli_x, np.eye(2**5))
    elif meas_out == '1000':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_x, np.eye(2**4)))
    elif meas_out == '1100':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_x, np.eye(2**3)))    
    elif meas_out == '0110':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_x, np.eye(2**2)))
    elif meas_out == '0011':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_x, np.eye(2**1)))
    elif meas_out == '1010':
        U_cor = np.kron(pauli_z, np.eye(2**5))
    elif meas_out == '0101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_z, np.eye(2**4)))
    elif meas_out == '0010':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_z, np.eye(2**3)))
    elif meas_out == '1001':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_z, np.eye(2**2)))
    elif meas_out == '0100':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_z, np.eye(2**1)))
    elif meas_out == '1011':
        U_cor = np.kron(pauli_y, np.eye(2**5))
    elif meas_out == '1101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_y, np.eye(2**4)))
    elif meas_out == '1110':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_y, np.eye(2**3)))
    elif meas_out == '1111':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_y, np.eye(2**2)))
    else:  # meas_out == '0111':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_y, np.eye(2**1)))
        
    # The final state
    final_state = U_cor @ outstate @ U_cor.conj().T
    
    return final_state


# 5. About coding -- get the state after Ug encoding
- 5-qubit code encoding algorithm
- The order of the space: 4 meas * 5 code * 1 anc

In [10]:
# Some vectors and matrices used in Encoding
# 5-qubit Encoding
# |0_L> = 1/4*(00000 + 10010 + 01001 + 10100 + 01010 - 11011 - 00110 - 11000 - 11101 - 00011 - 11110 - 01111 - 10001 - 01100 - 10111 + 00101)
# |1_L> = 1/4*(11111 + 01101 + 10110 + 01011 + 10101 - 00100 - 11001 - 00111 - 00010 - 11100 - 00001 - 10000 - 01110 - 10011 - 01000 + 11010)
    # Note: the expression from wiki has some problems
    
# Generate |0_L>, |1_L>: code_0, code_1
# |0_L>，(np.ket)
code_0 = np.zeros([2**5, 1])
code_0_list_plus = ['00000', '10010', '01001', '10100', '01010', '00101']
code_0_list_minus = ['11011', '00110', '11000', '11101', '00011', '11110', '01111', '10001', '01100', '10111']
for i in range(len(code_0_list_plus)):
    code_0[TwoToTen(code_0_list_plus[i])] = 1/4
for i in range(len(code_0_list_minus)):
    code_0[TwoToTen(code_0_list_minus[i])] = -1/4
# |0_L><0_L| density matrix
code_0_density = code_0 @ code_0.conj().T

# |1_L>，(np.ket)
code_1 = np.zeros([2**5, 1])
code_1_list_plus = ['11111', '01101', '10110', '01011', '10101', '11010']
code_1_list_minus = ['00100', '11001', '00111', '00010', '11100', '00001', '10000', '01110', '10011', '01000']
for i in range(len(code_1_list_plus)):
    code_1[TwoToTen(code_1_list_plus[i])] = 1/4
for i in range(len(code_1_list_minus)):
    code_1[TwoToTen(code_1_list_minus[i])] = -1/4
# |1_L><1_L| density matrix
code_1_density = code_1 @ code_1.conj().T
    
# The Encoding isometry: V_Eori
# V_E0 = |0_L><0| + |1_L><1|
V_Eori = code_0 @ vec(0,1) + code_1 @ vec(1,1)

In [11]:
# Get the state after encoding:
# For covarient encoding: V_Ecov = (Ug)^\otimes 5 V_E0 Ug^dag
# Hilbert space order: 4 meas * 5 code * 1 anc

def state_afterencoding(Ug):
    # Input: Ug
    # Output: input_total (paddle.matrix), final state in "4 meas * 5 code * 1 anc"
    
    # (Ug)^\otimes 5
    Ug5 = Ug
    for i in range(4):
        Ug5 = np.kron(Ug5, Ug)  # (Ug)^\otimes 5
    # V_Ecov = (Ug)^\otimes 5 V_E0 Ug^dag
    V_Ecov = Ug5 @ V_Eori @ Ug.conj().T
    
    # Get the state after encoding
    # Initial state: 1 logical \otimes 1 ancilla
    logical_ini = bell_state
    # After encoding: state：input_code, input_total
    input_code = np.kron(V_Ecov, np.eye(2)) @ logical_ini  # code \otimes anc，(np.ket)
    input_code_density = input_code @ input_code.conj().T  # code \otimes anc，(np.matrix)
    input_total = np.kron(vec(0, 4).conj().T, input_code)  # add 4 measure qubits
    input_total = input_total * input_total.conj().T  # (np.matrix)
    input_total = paddle.to_tensor(input_total, dtype = 'complex128')  # change to paddle density
    
    return input_total


# 6. About coding -- after noise and Uh decoding, calculate the entanglement fidelity

In [12]:
def fidelity_hg(Ug, Uh, input_total, noise):
    # Input: Ug, Uh -- np.matrix，
            # input_total: state after encoding -- paddle.matrix
            # noise model
                # noise = 1，a code qubit pass through the completely depolarizing noise
                # noise = 2，a code qubit pass through the completely dephasing noise
    # Output: fidelityh: Entanglement error with Ug encoding and Uh decoding
    
    # 1. Get the state after noise -- state_afternoise (np.matrix)
    # The input is input_total (paddle.matrix):  4 mea \otimes 5 code \otimes 1 anc
    # Go through the noise channel
    cir_noise = UAnsatz(10)  # This code has 10 qubit -- 4 measure, 5 code, 1 ancilla
    if noise == 1:
        # a code qubit pass through the completely depolarizing noise, p = 3/4
        cir_noise.depolarizing(3/4, 5)
    if noise == 2:
        # a code qubit pass through the completely dephasing noise, p = 3/4
        cir_noise.phase_flip(1/2, 5)
    state_afternoise = cir_noise.run_density_matrix(input_total).numpy()  # numpy.matrix

    
    # 2. Pass through decoding: Dec_h = Uh @ Dec @ Uh5^dag
    # 2.1 Go through Uh5^dag
    Uh5 = Uh
    for i in range(4):
        Uh5 = np.kron(Uh5, Uh)  # get (Uh)^\otimes 5
    # Get state_afterUhdag: 4 mea \otimes 5 code \otimes 1 anc
    # Apply eye(16) \otimes Uh5 \otimes eye(2))
    state_afterUhdag = np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)) @ state_afternoise @ np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)).conj().T
    state_afterUhdag = paddle.to_tensor(state_afterUhdag, dtype = 'complex128')  # change to (paddle.matrix)
    # 2.2 Go through decoding: 
    cir = UAnsatz(10)
    stabilizer_meas(cir)
    final_total_density = cir.run_density_matrix(state_afterUhdag).numpy()  # (numpy.matrix)
    # 2.3 Correct after measure
    # There are 16 measure outcomes, for each we do different corrections
    # Get the probability and output state for each measure outcomes, store in lists
        # prob_meas：32 elements: 16 outcomes (string), and 16 probabilities
        # prob_output：16 elements: 16 output states (numpy.matrix)
    prob_meas = []
    prob_output = []
    for i in range(2**4):
        basis = TenToTwo(i, n=4)
        pr, meas_prob_basis = MeasFiveQ(final_total_density, i)  # measure the first 4 qubits
        prob_meas.append(basis+':')
        prob_meas.append(pr)
        prob_output.append(meas_prob_basis)
        
    # 3. Do correction for each measure outcome and calculate fidelity for each measurement outcome
    fidelityh = 0
    for i in range(2**4):
        # Get state_aftercori, state_afterVdagi, state_afterUhi
        state_aftercori = correct_accoring_tomeas(TenToTwo(i, n=4), prob_output[i])
        state_afterVdagi = np.kron(V_Eori.conj().T, np.eye(2)) @ state_aftercori @ np.kron(V_Eori, np.eye(2))
        state_afterUhi = np.kron(Uh, np.eye(2)) @ state_afterVdagi @ np.kron(Uh.conj().T, np.eye(2))
        # calculate fidelity for each measurement outcome hi
        fidelityhi = np.real(bell_state.conj().T @ state_afterUhi @ bell_state)
        fidelityh += fidelityhi * np.real(prob_meas[2*i + 1])
    fidelityh = fidelityh[0][0]
    
    return fidelityh

    
    

# Numerical experiment part
- erasure noise

In [ ]:
# For erasure noise

# Get the relation between entanglement fidelity and N
# ITR_i_hg = 400: Calculate the average entanglement fidelity, with 400 random Ug
    # 1. Randomly generate Ug
    # 2. Get Uh -- From From_g_get_hath_largeRF(N, d, Ug)
    # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh# 对erasure noise

d = 2
noise = 1  # completely depolarizing noise
ITR_i_hg = 225
    
# Finally we will get 3 lists: N_list, f_N_ave_list, f_N_var_list
N_list=[70, 75]  # [20, 30, 40, 50, 60, 70, 80, 90, 100]
f_N_ave_list = []
f_N_var_list = []

# For each N in N_list, calculate fidelity_ave, fidelity_var
for N in N_list:
    print('N:', N)
    time_start = time.time() 

    fidelityh_list =  []  # store fidelityh in each iteration in the list to calculate the average and variance
    for i_hg in range(ITR_i_hg):
        # 1. Randomly generate Ug
        Ug = U2_random_Euler_mod()
        # 2. Get Uh -- From From_g_get_hath_largeRF(N, d, Ug)
        Uh = From_g_get_hath_largeRF(N, d, Ug)
        # 3. Calculate the entanglement fidelity: 'fidelityh' with this Ug, Uh
        # 3.1 The state after encoding: (Ug)^\otimes 5 V_E0 Ug^dag
        input_total = state_afterencoding(Ug)
        # 3.2 The fidelityh with the above Ug, Uh
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # depolarizing noise
        fidelityh_list.append(fidelityh)
    # calculate the average and variance for this N
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)
    



In [ ]:
# Fit and plot -- for erasure

# 1. Use the following list -- change to worst-case error
#N_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. Plot original figure
plt.figure(1)
func1, = plt.plot(N_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with composite RF state',
    ], loc='best')

plt.show()

# 3. Fit
# The performance is similar as: err_N_ave \app (coe / N^2)
def func(x, coe):
    return coe / (x**2)
Fitx = N_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
# Least squares method
popt, pcov = curve_fit(func, Fitx, Fity)
# Get the fitting coefficient
coe = popt[0] 
yvals = func(Fitx, coe)
print('coe:', coe)
print('pcov:', pcov)
# Plot
Fitxnew = []
Fitynew = []
for i in range(N_list[0], N_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with composite RF state',
    ], loc='best')
plt.show()

